###Instalação das dependências do Projeto

Scrapy para fazer parsear o html da pagina do 156 e coletar todas as urls dos csvs

Duckdb para para conseguir ler os arquivos de forma otimizada

In [1]:
!pip install -q scrapy
!pip install -q duckdb
!pip install -q pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.2 MB/s eta 0:00:00


In [5]:
import requests
from scrapy import Selector
from urllib.parse import urljoin
import pandas as pd
import requests
from glob import glob
from IPython.display import clear_output
import os

### Scraping:

A Etapa abaixo, realiza a raspagem do site da prefeitura, coletando todas as urls dos CSVs

In [2]:
def fetch_csv_urls():
    url = 'http://dados.prefeitura.sp.gov.br/dataset/dados-do-sp156'
    response = requests.get(url)
    print(response.status_code)
    sel = Selector(text=response.text)
    csv_links = sel.css('a::attr(href)').re(r'.*\.csv$')
    csv_links = [urljoin(response.url, link) for link in csv_links]
    return csv_links

if __name__ == "__main__":
    csv_urls = fetch_csv_urls()
    for url in csv_urls:
        print(url)
        with open('urls.txt', 'a') as file:
            file.write(url + '\n')


200
http://dados.prefeitura.sp.gov.br/dataset/0aecfa2b-aa3a-40d4-8183-0d4351b7fd0a/resource/2d020379-fa40-4dc0-8d17-add74b119550/download/arquivofinal2tri2024.csv
http://dados.prefeitura.sp.gov.br/dataset/0aecfa2b-aa3a-40d4-8183-0d4351b7fd0a/resource/787c054d-3e77-46c8-8713-e4e26eb2dd55/download/arquivofinal1tri2024.csv
http://dados.prefeitura.sp.gov.br/dataset/0aecfa2b-aa3a-40d4-8183-0d4351b7fd0a/resource/33d41278-02de-417a-a99a-5e01d3d87952/download/arquivofinal4tri2023.csv
http://dados.prefeitura.sp.gov.br/dataset/0aecfa2b-aa3a-40d4-8183-0d4351b7fd0a/resource/b01a9038-969e-4630-bd3a-4abe188b7259/download/arquivofinal3tri2023.csv
http://dados.prefeitura.sp.gov.br/dataset/0aecfa2b-aa3a-40d4-8183-0d4351b7fd0a/resource/7978d3ba-d379-44db-993b-61b9ff406435/download/arquivofinal2tri2023.csv
http://dados.prefeitura.sp.gov.br/dataset/0aecfa2b-aa3a-40d4-8183-0d4351b7fd0a/resource/d3043e41-4270-47b5-9922-9e35edd10950/download/arquivofinal1tri2023.csv
http://dados.prefeitura.sp.gov.br/dataset/

###Download dos arquivos CSV

Neste momento iremos realizar o download de todos os arquivos do tipo CSV e vamos converter para parquet, que é um formato mais otimizado.

In [ ]:
# Função para baixar o CSV
def download_csv(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
    else:
        print(f"Erro ao baixar {url}: Status {response.status_code}")


# Função para processar as URLs
def process_urls(file_with_urls):
    # Lê as URLs do arquivo de texto
    with open(file_with_urls, 'r') as file:
        urls = file.read().splitlines()

    for url in urls:
        file_name = os.path.basename(url)
        csv_path = f"downloads/{file_name}"
        download_csv(url, csv_path)
        print(f"download {file_name}")

file_with_urls = 'urls.txt'
os.makedirs('downloads', exist_ok=True)
process_urls(file_with_urls)


In [7]:
def csv_to_parquet():
    for path in glob('downloads/*.csv'):

        df = pd.read_csv(path, encoding='latin-1', sep=';')

        # Preenchendo valores NaN com padrões adequados
        df.fillna({'Data de abertura': '0000-00-00 00:00:00',
                   'Canal': '',
                   'Tema': '',
                   'Assunto': '',
                   'Serviço': '',
                   'Logradouro': '',
                   'CEP': '',
                   'Subprefeitura': '',
                   'Distrito': '',
                   'Latitude': 0.00,
                   'Longitude': 0.00,
                   'Data do Parecer': '0000-00-00 00:00:00',
                   'Status da solicitação': '',
                   'Orgão': '',
                   'Data': '0000-00-00',
                   'Prazo Atendimento': '',
                   'Qualidade Atendimento': '',
                   'Atendeu Solicitação': ''}, inplace=True)

        # Convertendo as colunas de data para datetime
        df['Data de abertura'] = pd.to_datetime(df['Data de abertura'], errors='coerce')
        df['Data do Parecer'] = pd.to_datetime(df['Data do Parecer'], errors='coerce')

        # Tratamento de colunas com valores numéricos e strings
        try:
            df['Número'] = df['Número'].astype(str).fillna('')
        except KeyError:
            df['Número'] = df['Numero'].astype(str).fillna('')

        try:
            df['Orgão'] = df['Orgão'].astype(str).fillna('')
        except KeyError:
            df['Orgão'] = df['Orgao'].astype(str).fillna('')

        try:
            df['Nível'] = df['Nível'].astype(str).fillna('')
        except KeyError:
            df['Nível'] = df['Nivel'].astype(str).fillna('')

        try:
            df['Atendeu Solicitação'] = df['Atendeu Solicitação'].astype(str).fillna('')
        except KeyError:
            df['Atendeu Solicitação'] = df['Atendeu Solicitacao'].astype(str).fillna('')

        df['CEP'] = df['CEP'].astype(str)
        df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce').fillna(0.00).astype(float)
        df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce').fillna(0.00).astype(float)

        # Criando colunas para ano e mês da Data de abertura
        df['Ano'] = df['Data de abertura'].dt.year
        df['Mes'] = df['Data de abertura'].dt.month

        # Selecionando as colunas que queremos
        df = df[['Data de abertura', 'Canal', 'Tema', 'Assunto',
                 'Serviço', 'Logradouro', 'Número', 'CEP',
                 'Latitude', 'Longitude', 'Data do Parecer',
                 'Status da solicitação', 'Orgão', 'Data',
                 'Nível', 'Prazo Atendimento', 'Qualidade Atendimento',
                 'Atendeu Solicitação', 'Ano', 'Mes']]

        # Gerando o caminho do parquet e particionando por ano e mês (formato Hive)
        parquet_path = f"database/{path.split('/')[-1].replace('.csv', '')}"

        df.to_parquet(parquet_path, index=False, partition_cols=['Ano', 'Mes'])
        clear_output(wait=True)

csv_to_parquet()


<ipython-input-7-d3ff3893aebe>:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, encoding='latin-1', sep=';')


In [14]:
!du -m parquets/* | sort -nr

23	parquets/arquivofinal1tri2021.parquet
22	parquets/arquivofinal2tri2023.parquet
19	parquets/arquivofinal1tri2024.parquet
19	parquets/arquivofinal1tri2023.parquet
17	parquets/arquivofinal1tri2022.parquet
16	parquets/arquivofinal6bi2020.parquet
16	parquets/arquivofinal4tri2023.parquet
16	parquets/arquivofinal2tri2024.parquet
15	parquets/arquivofinal3tri2023.parquet
15	parquets/arquivofinal3tri2021.parquet
14	parquets/arquivofinal4tri2022.parquet
14	parquets/arquivofinal4tri2021.parquet
14	parquets/arquivofinal2tri2022.parquet
13	parquets/arquivofinal3tri2022.parquet
12	parquets/arquivofinal1tri2019.parquet
11	parquets/arquivofinal2tri2019.parquet
10	parquets/arquivofinal4tri2019.parquet
10	parquets/arquivofinal3tri2019.parquet
10	parquets/arquivofinal1tri2016.parquet
9	parquets/arquivofinal5bi2020.parquet
9	parquets/arquivofinal4tri2018.parquet
9	parquets/arquivofinal4bi2020.parquet
9	parquets/arquivofinal1tri2018.parquet
9	parquets/arquivofinal1tri2017.parquet
9	parquets/arquivofinal1

In [33]:
query = """
    SELECT
      MIN("Data de abertura")
    FROM read_parquet('database/*/*/*/*.parquet')
"""
duckdb.sql(query).show()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────────────────────┐
│ min("Data de abertura") │
│        timestamp        │
├─────────────────────────┤
│ 2015-01-01 00:01:46     │
└─────────────────────────┘

